In [1]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math
import h5py
import numpy as np 
from multiprocessing import Pool
import os
from astropy.table import Table, vstack

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=50, 
    zl_max=2.5, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 17500.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 100000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [2]:
Ndraw

8991

In [3]:
current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    table = Table()

    fn = h5py.File(f"./samples/lenses_{count_draw}.hdf5", "r")
    src_thetaE = fn['source/thetaE'][()] #shape: [nsrc, n_ideal_lens]
    SNR = fn['Obs/SNR'][()] #shape: [nsrc, nband, n_ideal_lens]
    mu = fn['Obs/magnification'][()] #shape: [nsrc, nband, n_ideal_lens]
    ring_cond = fn['Obs/ring_cond'][()] #shape: [nsrc, nband, n_ideal_lenses]
    obs_cond = fn['Obs/detect_cond'][()] #shape: [nsrc, nband, n_ideal_lenses, 3]
    for ii in range(src_thetaE.shape[0]):
        table[f'thetaE_s{ii}'] =  src_thetaE[ii, :]
        table[f'mass_s{ii}'] = fn['source/einstein_mass'][()][ii, :]
        table[f'z_s{ii}'] = fn['source']['z'][()][ii, :]
        table[f're_s{ii}'] = fn['source']['Re'][()][ii, :]
        table[f'q_s{ii}'] = fn['source']['q'][()][ii, :]
        table[f'pa_s{ii}'] = fn['source']['pa'][()][ii, :]
        table[f'x_s{ii}'] = fn['source']['xs'][()][ii, :]
        table[f'y_s{ii}'] = fn['source']['ys'][()][ii, :]
        table[f'mag_g_s{ii}'] = fn['source']['app_mag_g'][()][ii, :]
        table[f'mag_r_s{ii}'] = fn['source']['app_mag_r'][()][ii, :]
        table[f'mag_i_s{ii}'] = fn['source']['app_mag_i'][()][ii, :]
        table[f'mag_z_s{ii}'] = fn['source']['app_mag_z'][()][ii, :]

        table[f'SNR_g{ii}'] = SNR[ii,0,:] #shape: [n_ideal_lens]
        table[f'SNR_r{ii}'] = SNR[ii,1,:]
        table[f'SNR_i{ii}'] = SNR[ii,2,:]
        table[f'SNR_z{ii}'] = SNR[ii,3,:]
        table[f'SNR_stack{ii}'] = SNR[ii,-1,:]

        SNR_griz = SNR[ii,0:-1,:] #shape: [4-color, n_ideal_lens]
        indices = np.argmax(SNR_griz, axis=0)
        table[f'best_band{ii}'] = np.array(['g', 'r', 'i', 'z'])[indices] 
        
        table[f'mu{ii}'] = mu[ii,-1,:] #shape: [n_ideal_lens]
        table[f'if_ring{ii}'] = ring_cond[ii,-1,:]

        #stack detection condition
        this_obs_cond = obs_cond.astype('int') #shape: [nsrc, nband, n_ideal_lenses, 3]
        this_obs_cond = this_obs_cond[ii, -1, :, :] #src-ii, band-stack, shape: [n_ideal_lens, 3-condition]
        this_obs_cond = np.sum(this_obs_cond, axis=1) #shape: [n_ideal_lens]
        bool_cond = (this_obs_cond==3)
        table[f'if_obs_stack{ii}'] = bool_cond #shape: [n_ideal_lens]

        #individual band
        this_obs_cond = obs_cond.astype('int')
        this_obs_cond = this_obs_cond[:, :-1, :, :] #remove stack band; shape: [nsrc, n_single_nband, n_ideal_lenses, 3-condition]
        cond = np.sum(this_obs_cond, axis=3)[ii,:,:] #detection condition for ii-th source: [nband, nlens]
        bool_cond = (cond==3).astype('int') #cond: [nband, n_ideal_lens]
        bool_cond = (np.sum(bool_cond, axis=0)>0) #bool_cond" [n_ideal_lens]
        table[f'if_obs_single{ii}'] = bool_cond

    table['vdisp_l'] = fn['deflector/vdisp'][()] #shape: [n_ideal_lens]
    table['re_l'] = fn['deflector/Re'][()]
    table['q_l']= fn['deflector/q'][()]
    table['z_l'] = fn['deflector/z'][()]
    table['mag_g_l'] = fn['deflector/app_mag_g'][()]
    table['mag_r_l'] = fn['deflector/app_mag_r'][()]
    table['mag_i_l'] = fn['deflector/app_mag_i'][()]
    table['mag_z_l'] = fn['deflector/app_mag_z'][()]

    fn.close()

    detect_conds = [table[f'if_obs_stack{i}'].data for i in range(src_thetaE.shape[0])] 
    detect_conds = np.vstack(detect_conds) #shape: [nsrc, n_ideal_lens]
    n_single_src_lens = np.count_nonzero(detect_conds.sum(axis=0)==1)
    n_double_src_lens = np.count_nonzero(detect_conds.sum(axis=0)==2)
    n_ideal_lens = src_thetaE.shape[1]
    table['noise_seed'] = np.arange(n_ideal_lens, dtype='int')
    
    return n_single_src_lens, n_double_src_lens, n_ideal_lens, table

n_single_src_lens, n_double_src_lens, n_ideal_lens, table = return_detected(0, stack=True)
print('111', n_single_src_lens, n_double_src_lens, n_ideal_lens)

111 0 0 23


In [4]:
pool = Pool(processes=128)
vars = list(zip(list(range(Ndraw)), [True]*Ndraw))
results = pool.starmap(return_detected, vars)

n_single_src_lens = [item[0] for item in results]
n_double_src_lens = [item[1] for item in results]
n_ideal_lens = [item[2] for item in results]

n_single_src_lens = sum(n_single_src_lens)
n_double_src_lens = sum(n_double_src_lens)
n_ideal_lens = sum(n_ideal_lens)

tables = [item[3] for item in results]
stacked_table = vstack(tables)

print('results', n_single_src_lens, n_double_src_lens, n_ideal_lens)
print(n_single_src_lens/n_ideal_lens)
print(n_double_src_lens/n_single_src_lens, n_single_src_lens/n_double_src_lens)

results 12643 245 172509
0.07328892985293521
0.01937831210946769 51.60408163265306


In [5]:
print(n_ideal_lens/n_single_src_lens, n_ideal_lens/n_double_src_lens)

13.644625484457803 704.1183673469387


In [11]:
mask0 = stacked_table['if_obs_stack0'].data
table_detect0 = stacked_table[mask0]

mask1 = stacked_table['if_obs_stack1'].data
table_detect1 = stacked_table[mask1]

mask_one = (stacked_table['if_obs_stack0'].data | stacked_table['if_obs_stack1'].data)
table_detect_one = stacked_table[mask_one]

mask_two = (stacked_table['if_obs_stack0'].data & stacked_table['if_obs_stack1'].data)
table_detect_two = stacked_table[mask_two]

print(len(table_detect0), len(table_detect1), len(table_detect_one), len(table_detect_two))
table_detect_two.write("catalog_csv/csst_double_src_wf_stack.csv", overwrite=True)

6592 6541 12888 245


In [12]:
mask0 = stacked_table['if_obs_single0'].data
table_detect0 = stacked_table[mask0]

mask1 = stacked_table['if_obs_single1'].data
table_detect1 = stacked_table[mask1]

mask_one = (stacked_table['if_obs_single0'].data | stacked_table['if_obs_single1'].data)
table_detect_one = stacked_table[mask_one]

mask_two = (stacked_table['if_obs_single0'].data & stacked_table['if_obs_single1'].data)
table_detect_two = stacked_table[mask_two]

print(len(table_detect0), len(table_detect1), len(table_detect_one), len(table_detect_two))
table_detect_two.write("catalog_csv/csst_double_src_wf_single.csv", overwrite=True)

3230 3193 6355 68
